In [1]:
import pystan
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
#### Preprocecing ###

In [3]:
Confirmed = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [4]:
df = Confirmed.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
confirmed = df

In [5]:
contries = confirmed.max() >= 1000
confirmed = confirmed.loc[:, contries]

In [6]:
df = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
df = df.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
recovered = df.loc[:, contries]


In [7]:
df = pd.read_csv("../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
df = df.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
death = df.loc[:, contries]


In [8]:
removed = recovered + death

In [9]:
### Data Preparation ###

In [10]:
country = 'Korea'
I0 = (confirmed - removed)[country].values
R0 = recovered[country].values
D = death[country].values
N = 5000_0000
repeat=10
iter=1000

In [11]:
data = {'T': I0.shape[0], 'N': N, 'I0': I0, 'R0': R0, 'D': D}

In [12]:
### Model ###

In [13]:
sm = pystan.StanModel(file="turzin.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_50906100ea65def1bb79b367af098c95 NOW.


In [ ]:
# Train the model and generate samples
fit = sm.sampling(data=data, iter=iter)

In [ ]:
### Simulation and Visualization functions ###

In [ ]:
T = S.shape[0]
N = S.shape[1]
epoch = pd.to_datetime('2020-01-22')

In [ ]:
def simulate(l0, beta0, beta1, theta, gamma, delta, q1, q2, 
             theta_q, q_date, sigma_S, sigma_R, sigma_I0, sigma_R0, sigma_D):
    susceptible = N
    infected = l0 + l0 * np.sqrt(sigma_I) * np.random.randn(1)
    recovered0 = 0
    recovered = 0
    deaeth = 0
    infected_list = []
    recovered_list = []
    death_list = []
    for d in range(T):
        infected_list.append(infected)
        recovered_list.append(recovered0)
        death_list.append(death)
        b = beta1 + (beta0 - beta1) * (1 - 1 / (1 + np.exp(-theta * (t - b_date)))); #transmission rate
        q = q1 + (q2 - q1) * (1 - 1 / (1 + np.exp(-theta_q * (t - q_date)))) # observation rate
        susceptible = susceptible - b * susceptible * infected / N 
                + susceptible * np.sqrt(sigma_S) * np.random.randn(1)
        infected = infected + b * susceptible * infected / N -
            (gamma + delta) * infected +
            infected * np.sqrt(sigma_I) * np.random.randn(1)
        recovered = recovered + gamma * infected 
        
        
        infected_list.append(cumulative - recovered)
        cumulative_list.append(cumulative)
        
    infected = pd.DataFrame(infected_list[L:], 
                            columns=confirmed.columns,
                            index=pd.date_range(start, periods=horizen, freq='D'))
    cumulative = pd.DataFrame(cumulative_list[L:], 
                            columns=confirmed.columns,
                            index=pd.date_range(start, periods=horizen, freq='D'))
    return infected, cumulative

In [ ]:
def draw_infected(fit, start, horizen, country):
    a_s = fit['a']
    p_s = fit['p']
    c_s = fit['c']
    sigma_S_s = fit['sigma_S']
    sigma_R_s = fit['sigma_R']
    n = a_s.shape[0]
    data = pd.DataFrame()
    for i in range(repeat):
        a = a_s[np.random.randint(n)]
        p = p_s[np.random.randint(n)]
        c = c_s[np.random.randint(n)]
        sigma_S = sigma_S_s[np.random.randint(n)]
        sigma_R = sigma_R_s[np.random.randint(n)]
        infected, cumulative = simulate(a, p, c, sigma_S, sigma_R, start, horizen)
        data = pd.concat([data, infected[country]], axis=1)
    graph = pd.DataFrame(index=pd.date_range(epoch - pd.Timedelta(days=L), 
                                             start + pd.Timedelta(days=horizen), freq='D'))
    real = (confirmed - removed)[country]
    median = data.median(axis=1)
    upper = data.quantile(q=0.75, axis=1)
    lower = data.quantile(q=0.25, axis=1)
    graph['Real'] = real
    graph['Median'] = median
    graph['Upper'] = upper
    graph['Lower'] = lower
    sns.relplot(kind="line", ci='sd', data=graph, aspect=2)

In [ ]:
### Validation ###

In [ ]:
draw_infected(fit, pd.to_datetime('2020-02-01'), 60, 'China')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-5'), 60, 'Italy')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'Iran')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'Norway')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'Korea, South')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'Japan')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'France')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'Germany')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-05'), 60, 'US')

In [ ]:
### Prediction ###

In [ ]:
recent = confirmed.index[-1]


In [ ]:
draw_infected(fit, recent, 60, 'China')

In [ ]:
draw_infected(fit, recent, 60, 'Italy')

In [ ]:
draw_infected(fit, recent, 60, 'Norway')

In [ ]:
draw_infected(fit, recent, 60, 'Korea, South')

In [ ]:
draw_infected(fit, recent, 60, 'Iran')

In [ ]:
draw_infected(fit, recent, 60, 'Japan')

In [ ]:
draw_infected(fit, recent, 60, 'France')

In [ ]:
draw_infected(fit, recent, 60, 'Germany')

In [ ]:
draw_infected(fit, recent, 60, 'US')

In [ ]:
def draw_infected_world(fit, start, horizen):
    a_s = fit['a']
    p_s = fit['p']
    c_s = fit['c']
    sigma_S_s = fit['sigma_S']
    sigma_R_s = fit['sigma_R']
    n = a_s.shape[0]
    data = pd.DataFrame()
    cum = []
    for i in range(repeat):
        a = a_s[np.random.randint(n)]
        p = p_s[np.random.randint(n)]
        c = c_s[np.random.randint(n)]
        sigma_S = sigma_S_s[np.random.randint(n)]
        sigma_R = sigma_R_s[np.random.randint(n)]
        infected, cumulative = simulate(a, p, c, sigma_S, sigma_R, start, horizen)
        data = pd.concat([data, infected.sum(axis=1)], axis=1)
        cum.append(cumulative.sum(axis=1).iloc[-1])
    cum = pd.Series(cum)
    graph = pd.DataFrame(index=pd.date_range('2020-01-22', start + pd.Timedelta(days=horizen)))
    real = (confirmed - removed).sum(axis=1)
    median = data.median(axis=1)
    upper = data.quantile(q=0.75, axis=1)
    lower = data.quantile(q=0.25, axis=1)
    graph['Real'] = real
    graph['Median'] = median
    graph['Upper'] = upper
    graph['Lower'] = lower
    sns.relplot(kind="line", ci='sd', data=graph, aspect=2);
    print("Upper:{}, Median:{}, Lower:{}".format(cum.quantile(q=0.75), cum.median(), cum.quantile(q=0.25)))

In [ ]:
draw_infected_world(fit, recent, 365)

In [ ]:
draw_infected(fit, recent, 60, 'Spain')

In [ ]:
confirmed.columns

In [ ]:
pystan.check_hmc_diagnostics(fit)

In [ ]:
draw_infected(fit, recent, 60, 'United Kingdom')

In [ ]:
draw_infected(fit, recent, 60, 'Canada')

In [ ]:
draw_infected(fit, recent, 60, 'Malaysia')

In [ ]:
sns.distplot(fit['p'][:,confirmed.columns.get_loc('Japan')])


In [ ]:
n = confirmed.columns.get_loc('China')
sns.distplot(fit['c'][n,n])

In [ ]:
def draw_infected_sums(fit, start, horizen):
    a_s = fit['a']
    p_s = fit['p']
    c_s = fit['c']
    sigma_S_s = fit['sigma_S']
    sigma_R_s = fit['sigma_R']
    n = a_s.shape[0]
    data = pd.DataFrame()
    for i in range(repeat):
        a = a_s[np.random.randint(n)]
        p = p_s[np.random.randint(n)]
        c = c_s[np.random.randint(n)]
        sigma_S = sigma_S_s[np.random.randint(n)]
        sigma_R = sigma_R_s[np.random.randint(n)]
        _, cumulative = simulate(a, p, c, sigma_S, sigma_R, start, horizen)
        data = pd.concat([data, cumulative.iloc[-1]], axis=1)
    melted = pd.melt(data.T).rename(columns={'value':'Infected'})
    my_order = melted.groupby(by=['Country/Region'])['Infected'].median().sort_values(ascending=False).index
    plt.figure(figsize=(10,20))
    sns.boxplot(data=melted, y = 'Country/Region', x='Infected', showfliers=False, order=my_order)
    table = pd.DataFrame({'Lower':data.quantile(q=0.25, axis=1), 
                           'Median':data.median(axis=1), 
                           'Upper':data.quantile(0.75, axis=1)})
    return table

In [ ]:
data = draw_infected_sums(fit, recent, 365)

In [ ]:
data

In [ ]:
draw_infected(fit, pd.to_datetime('2020-02-15'), 60, 'Korea, South')

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-01'), 60, 'Korea, South')

In [ ]:
df = Confirmed.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))

In [ ]:
df['Thailand']

In [ ]:
np.sqrt([1, 1, 1])

In [ ]:
np.sqrt([1, 1, 1]) * np.random.randn(3)

In [ ]:
draw_infected(fit, pd.to_datetime('2020-03-15'), 60, 'Turkey')